<a href="https://colab.research.google.com/github/Denev6/CapStone/blob/main/tdcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TDCN 이미지 예측 모델 구현

In [ ]:
import os
import gc
import warnings
from google.colab import drive

import numpy as np 
import pandas as pd
from tqdm.auto import tqdm, trange
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
)

In [ ]:
drive.mount("/content/drive")
warnings.simplefilter("ignore")

def clear():
    gc.collect()
    torch.cuda.empty_cache()

def join_path(*args):
    return os.path.join("/content/drive/MyDrive/Capstone", *args)

# 사용한 데이터 파일
TRAIN_CSV = [
    join_path("data", csv)
    for csv in ["feat_xtrain.csv", "pose_xtrain.csv", "ytrain.csv"]
]
TEST_CSV = [
    join_path("data", csv) for csv in ["feat_xtest.csv", "pose_xtest.csv", "ytest.csv"]
]

device = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8
EPOCHS = 100
LEARNING_RATE = 5e-4
SMOOTHING = 0.0
MODEL_PATH = join_path("tdcn.pth")

Mounted at /content/drive


In [ ]:
class EarlyStopping(object):
    def __init__(self, patience=2, save_path="model.pth", eps=1e-6):
        self._min_loss = np.inf
        self._patience = patience
        self._path = save_path
        self._eps = eps
        self.__counter = 0

    def should_stop(self, model, loss):
        if loss < self._min_loss:
            self._min_loss = loss
            self.__counter = 0
            torch.save(model.state_dict(), self._path)
        elif loss > self._min_loss + self._eps:
            self.__counter += 1
            if self.__counter >= self._patience:
                return True
        return False
   
    def load(self, model):
        model.load_state_dict(torch.load(self._path))
        return model
    
    @property
    def counter(self):
        return self.__counter

# Dataset

In [ ]:
class CustomDataset(Dataset):
    """데이터 처리"""
    def __init__(self, x1_file, x2_file, y_file, mode=None):
        x1_df = pd.read_csv(x1_file)  # landmark data
        x2_df = pd.read_csv(x2_file)  # pose data
        y_df = pd.read_csv(y_file)

        x1 = x1_df.values
        x2 = x2_df.values

        if mode == "train":
            # 데이터 전처리 문제로 인덱스가 포함되어 있음
            y = y_df.iloc[:, 1].values
        else:
            y = y_df.iloc[:, 0].values

        self.x1_data = torch.FloatTensor(x1)
        self.x2_data = torch.FloatTensor(x2)
        self.y_data = torch.IntTensor(y)

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, index):
        return (
            self.x1_data[index * 5000 : (index + 1) * 5000],
            self.x2_data[index * 5000 : (index + 1) * 5000],
        ), self.y_data[index]

In [ ]:
# 데이터셋
training_data = CustomDataset(*TRAIN_CSV, mode="train")
test_data = CustomDataset(*TEST_CSV)

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

# Dilated Conv Block

In [ ]:
class DilatedConvBlock(nn.Module):
    """모델 내의 DCN Block"""
    def __init__(self, has_BN=False, batch_size=8):
        super(DilatedConvBlock, self).__init__()
        self._c = 1
        self._has_BN = has_BN

        self.dilated_conv1 = self.dilated_conv(1)
        self.dilated_conv2 = self.dilated_conv(2)
        self.dilated_conv3 = self.dilated_conv(4)
        self.conv1d = nn.Conv2d(self._c, self._c, kernel_size=(1, 1))
        self.BN = nn.BatchNorm2d(self._c, affine=True)

    def dilated_conv(self, d):
        return nn.Conv2d(
            self._c,
            self._c,
            kernel_size=(3, 3),
            stride=1,
            padding="same",
            dilation=d,
            bias=True,
            padding_mode="zeros",
        )

    def forward(self, x):
        x_1d = self.conv1d(x)

        # DCN with d=1
        x_2d_1 = self.dilated_conv1(x)
        x_2d_2 = self.dilated_conv1(x)
        x_2d = x_2d_1 + x_2d_2
        x_2d = F.elu(x_2d)

        # DCN with d=2
        x_2d_1 = self.dilated_conv2(x_2d)
        x_2d_2 = self.dilated_conv2(x_2d)
        x_2d = x_2d_1 + x_2d_2
        x_2d = F.elu(x_2d)

        # DCN with d=4
        x_2d_1 = self.dilated_conv3(x_2d)
        x_2d_2 = self.dilated_conv3(x_2d)
        x_2d = x_2d_1 + x_2d_2
        x_2d = F.elu(x_2d)

        x = x_1d + x_2d

        if self._has_BN:
            x = self.BN(x)
        return x

# TDCN + FWA + Prediction

In [ ]:
class PredictionModel(nn.Module):
    """모델 전체 구조"""
    def __init__(self, batch_size=8):
        super(PredictionModel, self).__init__()

        self.TDCN = nn.Sequential(
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=True, batch_size=BATCH_SIZE),
            nn.MaxPool2d(kernel_size=(2, 1), stride=None),
            DilatedConvBlock(has_BN=False),
        )
        self.attention = nn.Sequential(
            nn.Linear(142, 142),
            nn.ReLU(inplace=True),
            nn.Linear(142, 142),
            nn.Sigmoid(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(142, 64), nn.Linear(64, 64), nn.Linear(64, 2), nn.Softmax(dim=3)
        )

    def global_average_pooling(self, x):
        return torch.mean(x, dim=2)

    def FWA(self, x1, x2):
        x = torch.concat((x1, x2), dim=3)
        x_ = self.global_average_pooling(x)
        x_ = self.attention(x_)
        x_ = torch.unsqueeze(x_, 2)
        x = torch.mul(x, x_)

        return x

    def forward(self, x_landmark, x_pose):
        x_landmark = self.TDCN(x_landmark)
        x_pose = self.TDCN(x_pose)
        x = self.FWA(x_landmark, x_pose)
        score = self.classifier(x)
        score = score.mean(dim=2)
        return score

# Train

In [ ]:
loss_fn = nn.CrossEntropyLoss(label_smoothing=SMOOTHING)
model = PredictionModel(BATCH_SIZE)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [ ]:
# 모델 전체 구조
model.eval()

PredictionModel(
  (TDCN): Sequential(
    (0): DilatedConvBlock(
      (dilated_conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (dilated_conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(2, 2))
      (dilated_conv3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(4, 4))
      (conv1d): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
      (BN): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (2): DilatedConvBlock(
      (dilated_conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (dilated_conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(2, 2))
      (dilated_conv3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, dilation=(4, 4))
      (conv1d): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
      (BN)

In [ ]:
def train(train_loader, test_loader, model, loss_fn, optimizer):
    model.to(device) 
    loss_fn.to(device)
    model.zero_grad()

    num_batches = len(train_loader)
    early_stopper = EarlyStopping(patience=3, save_path=MODEL_PATH)

    epoch_progress = trange(1, EPOCHS + 1)
    tqdm.write("\nEpoch | Train Loss | Test Loss")
    tqdm.write("-" * 30)
    
    for epoch in epoch_progress:
        model.train()
        train_loss = 0
        for (x1, x2), label in train_loader:
            x1 = x1.to(device)
            x2 = x2.to(device)
            label = label.to(device)
            
            pred = model(
                x1.unsqueeze(0).permute(1, 0, 2, 3), x2.unsqueeze(0).permute(1, 0, 2, 3)
            )
            loss = loss_fn(pred.squeeze(1), label.long())
            train_loss += loss.item()
            test_loss = test(test_loader, model, loss_fn)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= num_batches
    
        tqdm.write(f"{epoch:5} | {train_loss:10.5f} | {test_loss:9.5f}")
        
        if early_stopper.should_stop(model, test_loss):
            tqdm.write(f"--EarlyStopping: [Epoch: {epoch - early_stopper.counter}]")
            break

    model = early_stopper.load(model)
    return model


def test(test_loader, model, loss_fn, return_metrics=False):
    model.eval()
    num_batches = len(test_loader)
    test_loss = 0
    true_labels = list()
    pred_values = list()

    with torch.no_grad():
        for (x1, x2), label in test_loader:
            x1 = x1.to(device)
            x2 = x2.to(device)
            label = label.to(device)

            pred = model(
                x1.unsqueeze(0).permute(1, 0, 2, 3), x2.unsqueeze(0).permute(1, 0, 2, 3)
            )
            pred = pred.squeeze()

            test_loss += loss_fn(pred.squeeze(1), label.long()).item()

            if return_metrics:
                true_labels += label.detach().cpu().numpy().tolist()
                pred_values += pred.argmax(-1).detach().cpu().numpy().tolist()

    if not return_metrics:
        # 학습 과정에서는 Loss 값만 확인합니다.
        test_loss /= num_batches
        return test_loss

    else:
        # 학습이 종료되고 성능 평가 지표를 확인합니다.
        accuracy = accuracy_score(true_labels, pred_values)
        f1 = f1_score(true_labels, pred_values)
        f1_macro = f1_score(true_labels, pred_values, average="macro")
        recall = recall_score(true_labels, pred_values)
        precision = precision_score(true_labels, pred_values)
        matrix = confusion_matrix(true_labels, pred_values).ravel()

        return {
            "accuracy": accuracy,
            "f1": f1,
            "f1-macro": f1_macro,
            "recall": recall,
            "precision": precision,
            "loss": test_loss,
            "matrix": matrix,
        }

In [ ]:
# 모델 학습
model = train(train_dataloader, test_dataloader, model, loss_fn, optimizer)

  0%|          | 0/100 [00:00<?, ?it/s]


Epoch | Train Loss | Test Loss
------------------------------
    1 |    0.66002 |   0.63160
    2 |    0.63464 |   0.61037
    3 |    0.63082 |   0.60695
    4 |    0.62774 |   0.60519
    5 |    0.62728 |   0.60491
    6 |    0.62717 |   0.60461
    7 |    0.62912 |   0.60431
    8 |    0.62891 |   0.60449
    9 |    0.62722 |   0.60416
   10 |    0.62907 |   0.60437
   11 |    0.62986 |   0.60405
   12 |    0.62934 |   0.60450
   13 |    0.62856 |   0.60421
   14 |    0.62966 |   0.60422
--EarlyStopping: [Epoch: 11]


In [ ]:
# 모델 저장
torch.save(
    {
        "epoch": EPOCHS,
        "learning_rate": LEARNING_RATE,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
    },
    MODEL_PATH,
)

# Evaluation

```
# 저장된 모델 불러오기
loss_fn = nn.CrossEntropyLoss()

checkpoint = torch.load(join_path("tdcn.pth"))

model = PredictionModel(BATCH_SIZE).to(device)
model.load_state_dict(checkpoint["model_state"])

# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
# optimizer.load_state_dict(checkpoint["optimizer_state"])
```

In [ ]:
clear()
metrics = test(test_dataloader, model, loss_fn, return_metrics=True)

print(f"Accuracy:  {metrics['accuracy']:.3f}")
print(f"F1-score:  {metrics['f1']:.3f}")
print(f"F1-macro:  {metrics['f1-macro']:.3f}")
print(f"Recall:    {metrics['recall']:.3f}")
print(f"Precision: {metrics['precision']:.3f}")

print("-" * 30)
tn, fp, fn, tp = metrics["matrix"]
print(f"TN: {tn}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print(f"TP: {tp}")

Accuracy:  0.702
F1-score:  0.000
F1-macro:  0.412
Recall:    0.000
Precision: 0.000
------------------------------
TN: 33
FP: 0
FN: 14
TP: 0


In [ ]:
def show_probs(test_data, model, max=6):
    dataloader = DataLoader(test_data, batch_size=1, shuffle=True)

    model.eval()
    neg_max = max // 2
    pos_max = max - neg_max
    pos_count = 0
    neg_count = 0
    with torch.no_grad():
        for X, label in dataloader:
            x1, x2 = X
            x1, x2 = x1.to(device), x2.to(device)

            if label.item() == 0 and pos_count < pos_max:
                pos_count += 1
                label = label.item()
            elif label.item() == 1 and neg_count < neg_max:
                neg_count += 1
                label = label.item()
            elif pos_count + neg_count == max:
                break
            else:
                continue

            pred = model(
                x1.unsqueeze(0).permute(1, 0, 2, 3), x2.unsqueeze(0).permute(1, 0, 2, 3)
            )
            normal, abnormal = pred.squeeze()
            print(f"{label}: [{normal:.3f}  {abnormal:.3f}]")


# 예측된 확률: [우울증이 아닐 확률, 우울증일 확률]
show_probs(test_data, model, 10)

0: [0.897  0.103]
1: [0.890  0.110]
1: [0.885  0.115]
0: [0.897  0.103]
0: [0.893  0.107]
0: [0.892  0.108]
1: [0.886  0.114]
1: [0.886  0.114]
1: [0.887  0.113]
0: [0.887  0.113]
